<a href="https://colab.research.google.com/github/OscarBedford/MLCourse_Weekly_Exercises/blob/main/Exercise3_5_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

5. Use sklearn.linear_model.LogisticRegression(hyperparameter set to 1.0) to fit separate logistic predictive models based on 100, 200, 300, …, 900, 1000
brain regions of interest (see ‘n_rois’) based on the first 80% of the 100 structural brain scans to predict sex differences. Use the probabilistic prediction of LogReg (predict_proba function) from the training data from the 10 already obtained LogReg models to fit a LogReg stacking model to predict sex. Next, evaluate the final LogReg stacking model on the unseen final 20% of the 100 structural brain scans and print classification accuracy in XX.YY%.

In [113]:
!pip install nilearn
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [114]:
import numpy as np 
from nilearn import datasets 
from nilearn.maskers import NiftiLabelsMasker 
from nilearn.image import index_img 
import nibabel as nib

In [115]:
brain_data = datasets.fetch_oasis_vbm(n_subjects=100) 
yeo = datasets.fetch_atlas_schaefer_2018(n_rois=1000) 
masker = NiftiLabelsMasker(labels_img=yeo.maps, standardize=True, memory='nilearn_cache') 
input_variables = masker.fit_transform(brain_data.gray_matter_maps) 
output_variable = np.array(brain_data.ext_vars.mf == 'F', dtype=np.int) #gives 1 for females and 0 for males

/usr/local/lib/python3.7/dist-packages/nilearn/datasets/struct.py:774: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """


In [116]:
import matplotlib.pyplot as plt
import pandas as pd # we will import pandas as well for this exercise
from numpy import mean
from numpy import std
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.ensemble import StackingClassifier

In [117]:
# Let's make sure we have a 100x1000 dataset
print (input_variables.shape)

(100, 1000)


In [118]:
# Checking the output variable
output_variable

array([1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0])

In [163]:
# We create the variable with which we will iterate through the 10 groups of ROIs
ROIs = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

In [164]:
#We pre-allocate the variables that we're interested in storing
training_preds = [] # Here we will store the probability classes of the 10 training sets
test_preds = [] # Here we will store the probability classes of the 10 test sets

In [165]:
# We're ready to run the predict_proba model
for x in ROIs:
    brain_data = datasets.fetch_oasis_vbm(n_subjects=100) 
    yeo = datasets.fetch_atlas_schaefer_2018(n_rois=(x))  # We iterate thru the list of ROI values
    masker = NiftiLabelsMasker(labels_img=yeo.maps, standardize=True, memory='nilearn_cache') 
    input_variables = masker.fit_transform(brain_data.gray_matter_maps) 
    output_variable = np.array(brain_data.ext_vars.mf == 'F', dtype=np.int)

    X, y = input_variables, output_variable
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

    LR_proba = LogisticRegression(max_iter=1e4).fit(X_train,y_train) # We fit the LR learning model
    preds = LR_proba.predict_proba(X_train) # We define the training model and store its probability classifications as 'preds'
    training_preds.append(preds) # This is where we will iteratively store the scores for the training set probability class predictions
    score = LR_proba.score(X_train, y_train) # We will also export the score (...)
    score = np.multiply(score, 100) 
    print('Training accuracy: %.2f' % ((score))) # (...) and we will print it

    yhat = LR_proba.predict_proba(X_test) # We define the prediction model and store its prediction as "yhat"
    test_preds.append(yhat) # This is where we will store the test set class predictions
    scores = LR_proba.score(X_test, y_test) # We store the scores
    scores = np.multiply(scores, 100) 
    print('Test accuracy: %.2f' % ((scores))) # We print the scores

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
/usr/local/lib/python3.7/dist-packages/nilearn/datasets/struct.py:774: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)


Training accuracy: 100.00
Test accuracy: 55.00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
/usr/local/lib/python3.7/dist-packages/nilearn/datasets/struct.py:774: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)


Training accuracy: 100.00
Test accuracy: 70.00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
/usr/local/lib/python3.7/dist-packages/nilearn/datasets/struct.py:774: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)


...

/usr/local/lib/python3.7/dist-packages/nilearn/datasets/struct.py:774: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
/usr/local/lib/python3.7/dist-packages/nilearn/datasets/struct.py:774: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then r

Training accuracy: 100.00
Test accuracy: 80.00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
/usr/local/lib/python3.7/dist-packages/nilearn/datasets/struct.py:774: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)


Training accuracy: 100.00
Test accuracy: 80.00
Training accuracy: 100.00
Test accuracy: 75.00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys


In [219]:
# Before we can compute the stacking model, we need to reshape the lists we got
training_preds = np.array(training_preds) # Convert output to numpy array for ease of manipulation
test_preds = np.array(test_preds) # ditto
training_preds = np.delete(training_preds, 1, 2) # We delete column 2 (index 1) from the third axis (index 2) because it's redundant
test_preds =  np.delete(test_preds, 1, 2) #ditto

training_preds = np.squeeze(training_preds) # We eliminate the extraneous axis of size 1
test_preds = np.squeeze(test_preds) #ditto
training_preds = np.transpose(training_preds) # We transpose the array so that rows (observations) are first and columns (features) are second
test_preds = np.transpose(test_preds) #ditto

In [220]:
 # We're ready to compute the stacking (or meta-) model:   
LR_stack = LogisticRegression(max_iter=1e4).fit(training_preds, y_train) # We fit the LR learning model
score = LR_stack.score(training_preds, y_train) # We will also export the score (...)
score = np.multiply(score, 100) 
print('Meta-model training accuracy: %.2f' % ((score))) # (...) and we will print it
   
yhat_stack = LR_stack.predict_proba(test_preds) # We define the prediction model and store its prediction as "yhat"
scores = LR_stack.score(test_preds, y_test) # We store the test set scores (...)
scores = np.multiply(scores, 100) 
print('Meta-model test accuracy: %.2f' % ((scores))) # (...) and we print the them

Meta-model training accuracy: 100.00
Meta-model test accuracy: 75.00


Voilà! We obtain a test set classification accuracy of 75% for the stacking (or meta-) model. 